In [ ]:
import csv
from itertools import groupby

with open('../../data/toflit18_all_flows.csv', 'r') as data:
    key = lambda r : (r['origin_province'],r['customs_region'])
    exports_from_fr_to_marseille = sorted([row for row in csv.DictReader(data) if \
            row['year'] == "1789" and \
            row['best_guess_region_prodxpart'] == '1' and \
            row['partner_simplification'] == "Marseille" and \
            row['customs_region'] != 'Marseille' and \
            row['export_import'] == 'Exports'], key=key)
    
    data = []
    total_value = 0
    for (origine, exporter), flows in groupby(exports_from_fr_to_marseille, key= key):
        value = sum(float(flow['value']) for flow in flows if flow["value"] != '')
        total_value += value
        data.append({"value":value, 'origine':origine if origine != '' else 'inconnue', 'direction_ferme': exporter})

    print(total_value)

In [ ]:
data


In [ ]:
def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Origine par direction de fermes des imports pour Marseille en 1789",
    "width": 800,
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar"
    },
    "data": {
        "values": data
    },
    "encoding": {
        "row": {
                "field": "origine",  
                "header":{"labelOrient": "top"},
                "sort": {
                    "op": "sum",
                    "field": "value",
                    "order": "descending"
                    
                }
            },
        "x": {
            
            "field": "value",
            "type": "quantitative"
        },
        "y": {
            "field": "direction_ferme",
            "type": "nominal",
            "title": "",
            "axis": {
                "labelLimit": 500,
            },
            "sort": "-x"
            
        },
        "color": {
            "field": "direction_ferme",
            "type": "nominal",
            "title": "Direction de ferme exportatrice"
        },
       
        
    },
    "resolve": {"scale": {"y": "independent"}}

})